In [1]:
import os

In [2]:
from gymnasium.spaces import Box, Discrete

In [3]:
import ray
from ray import tune
from ray.rllib.algorithms.dqn import DQNConfig
from ray.rllib.algorithms.dqn.dqn_torch_model import DQNTorchModel
from ray.rllib.env import PettingZooEnv
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.torch_utils import FLOAT_MAX
from ray.tune.registry import register_env

/Users/drkp4/.pyenv/versions/anaconda3-2023.03/envs/rlenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-15 17:07:47,165	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-10-15 17:07:47,533	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-10-15 17:07:48,728	WARNING compression.py:16 -- lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
2023-10-15 17:07:48,749	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-10-15 17:07:49,2

In [4]:
from pettingzoo.classic import leduc_holdem_v4

In [5]:
torch, nn = try_import_torch()
torch, nn

(<module 'torch' from '/Users/drkp4/.pyenv/versions/anaconda3-2023.03/envs/rlenv/lib/python3.10/site-packages/torch/__init__.py'>,
 <module 'torch.nn' from '/Users/drkp4/.pyenv/versions/anaconda3-2023.03/envs/rlenv/lib/python3.10/site-packages/torch/nn/__init__.py'>)

In [6]:
class TorchMaskedActions(DQNTorchModel):
    """PyTorch version of above ParametricActionsModel."""

    def __init__(
        self,
        obs_space: Box,
        action_space: Discrete,
        num_outputs,
        model_config,
        name,
        **kw,
    ):
        DQNTorchModel.__init__(
            self, obs_space, action_space, num_outputs, model_config, name, **kw
        )

        obs_len = obs_space.shape[0] - action_space.n

        orig_obs_space = Box(
            shape=(obs_len,), low=obs_space.low[:obs_len], high=obs_space.high[:obs_len]
        )
        self.action_embed_model = TorchFC(
            orig_obs_space,
            action_space,
            action_space.n,
            model_config,
            name + "_action_embed",
        )

    def forward(self, input_dict, state, seq_lens):
        # Extract the available actions tensor from the observation.
        action_mask = input_dict["obs"]["action_mask"]

        # Compute the predicted action embedding
        action_logits, _ = self.action_embed_model(
            {"obs": input_dict["obs"]["observation"]}
        )
        # turns probit action mask into logit action mask
        inf_mask = torch.clamp(torch.log(action_mask), -1e10, FLOAT_MAX)

        return action_logits + inf_mask, state

    def value_function(self):
        return self.action_embed_model.value_function()

In [7]:
ray.init()

2023-10-15 17:07:51,119	INFO worker.py:1642 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.7.1


In [8]:
alg_name = "DQN"
ModelCatalog.register_custom_model("pa_model", TorchMaskedActions)
# function that outputs the environment you wish to register.

def env_creator():
    env = leduc_holdem_v4.env()
    return env


In [9]:
env_name = "leduc_holdem_v4"
register_env(env_name, lambda config: PettingZooEnv(env_creator()))


In [10]:
test_env = PettingZooEnv(env_creator())
obs_space = test_env.observation_space
act_space = test_env.action_space

In [11]:
config = (
    DQNConfig()
    .environment(env=env_name)
    .rollouts(num_rollout_workers=1, rollout_fragment_length=30)
    .training(
        train_batch_size=200,
        hiddens=[],
        dueling=False,
        model={"custom_model": "pa_model"},
    )
    .multi_agent(
        policies={
            "player_0": (None, obs_space, act_space, {}),
            "player_1": (None, obs_space, act_space, {}),
        },
        policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
    )
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    .debugging(
        log_level="DEBUG"
    )  # TODO: change to ERROR to match pistonball example
    .framework(framework="torch")
    .exploration(
        exploration_config={
            # The Exploration class to use.
            "type": "EpsilonGreedy",
            # Config for the Exploration class' constructor:
            "initial_epsilon": 0.1,
            "final_epsilon": 0.0,
            "epsilon_timesteps": 100000,  # Timesteps over which to anneal epsilon.
        }
    )
)

In [12]:
tune.run(
    alg_name,
    name="DQN",
    stop={"timesteps_total": 10000000},
    checkpoint_freq=10,
    config=config.to_dict(),
)

2023-10-15 17:07:52,024	INFO tune.py:645 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-10-15 17:07:52,068	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/Users/drkp4/.pyenv/versions/anaconda3-2023.03/envs/rlenv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/drkp4/.pyenv/versions/anaconda3-20

(pid=92610) lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
(pid=92610) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(pid=92610) 2023-10-15 17:07:55,143	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
(pid=92610) 2023-10-15 17:07:55,144	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
(DQN pid=92610) 2023-10-15 17:07:55,153	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/simple_q/` has been deprecated. Use `rllib_contrib/simple_q/` instead. This will raise an error in the future!
(DQN pid=92610) 2023-10-15 17:07:55,153	WARNING algorithm_config.py:672 -- Cannot create DQNConfig from given `config_dict`! Property __stdout_file__ not supported.
(RolloutWorker pid=92

(RolloutWorker pid=92616) [WARNING]: Illegal move made, game terminating with current player losing. 
(RolloutWorker pid=92616) obs['action_mask'] contains a mask of all legal moves that can be chosen.
(RolloutWorker pid=92616) [WARNING]: Illegal move made, game terminating with current player losing. 
(RolloutWorker pid=92616) obs['action_mask'] contains a mask of all legal moves that can be chosen.
(RolloutWorker pid=92616) [WARNING]: Illegal move made, game terminating with current player losing. 
(RolloutWorker pid=92616) obs['action_mask'] contains a mask of all legal moves that can be chosen.
(RolloutWorker pid=92616) [WARNING]: Illegal move made, game terminating with current player losing. 
(RolloutWorker pid=92616) obs['action_mask'] contains a mask of all legal moves that can be chosen.
(RolloutWorker pid=92616) [WARNING]: Illegal move made, game terminating with current player losing. 
(RolloutWorker pid=92616) obs['action_mask'] contains a mask of all legal moves that can b

(DQN pid=92610) 2023-10-15 17:08:00,627	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!
(DQN pid=92610) 2023-10-15 17:08:00,628	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(pid=92616) lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
(pid=92616) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=92616) 2023-10-15 17:07:58,343	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=92616) 2023-10-15 17:07:58,344	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!


(DQN pid=92610) 2023-10-15 17:08:00,654	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:08:00,833	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:08:00,837	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:08:00,925	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:08:00,928	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==


...

(DQN pid=92610) 2023-10-15 17:12:57,915	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:12:57,918	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:12:58,003	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:12:58,006	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:12:58,093	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:12:58,096	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:12:58,189	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:12:58,193	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:12:58,286	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:12:58,289	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:12:58,377	

(RolloutWorker pid=92616) [WARNING]: Illegal move made, game terminating with current player losing. 
(RolloutWorker pid=92616) obs['action_mask'] contains a mask of all legal moves that can be chosen.


(DQN pid=92610) 2023-10-15 17:12:59,351	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:12:59,354	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:12:59,493	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:12:59,496	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:12:59,582	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:12:59,585	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(RolloutWorker pid=92616) 2023-10-15 17:12:59,597	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_action_dist.get_torch_categorical_class_with_temperature` has been deprecated. Use `ray.rllib.models.torch.torch_distributions.TorchCategorical` instead. This will raise an error in the future!
(RolloutWorker pid=92616) 2023-10-15 17:12:59,597	WARNING deprecation.py:50 -- DeprecationWarning: `r

(RolloutWorker pid=92616) [WARNING]: Illegal move made, game terminating with current player losing. 
(RolloutWorker pid=92616) obs['action_mask'] contains a mask of all legal moves that can be chosen.


(DQN pid=92610) 2023-10-15 17:13:00,205	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:13:00,208	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:13:00,293	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:13:00,296	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:13:00,383	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:13:00,386	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:13:00,472	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:13:00,475	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:13:00,560	DEBUG train_ops.py:164 -- == sgd epochs for player_0 ==
(DQN pid=92610) 2023-10-15 17:13:00,563	DEBUG train_ops.py:164 -- == sgd epochs for player_1 ==
(DQN pid=92610) 2023-10-15 17:13:00,649	